In [ ]:
import numpy as np
import torch 
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
!pip install timm
import timm
import pandas as pd 
import random
import math

In [ ]:
class DenseNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.timm_model = timm.create_model('densenet201', pretrained=True, in_chans=1)
        self.fc = nn.Linear(1000, 30)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.timm_model(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

In [ ]:
class EfficientNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.timm_model = timm.create_model('tf_efficientnetv2_s', pretrained=True, in_chans=1)
        self.fc = nn.Linear(1000, 30)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.timm_model(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

In [ ]:
class MobileNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.timm_model = timm.create_model('mobilenetv3_large_100', pretrained=True, in_chans=1)
        self.fc = nn.Linear(1000, 30)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.timm_model(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

In [ ]:
class InceptionNextModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.timm_model = timm.create_model('inception_next_small', pretrained=True, in_chans=1)
        self.fc = nn.Linear(1000, 30)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.timm_model(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

In [ ]:
class MyDataset(data.Dataset):
    def __init__(self, mode, images, targets=None):
        super().__init__()
        self.mode = mode
        self.images = images
        self.targets = targets
    
    def transform(self, image, target=None):
        image = transforms.ToTensor()(image)
        if self.mode == 'train':
            p = random.random()
            if p > 0.5:
                image = transforms.GaussianBlur(5)(image)
                           
            p = random.random()
            if p > 0.5:
                mask = torch.rand(size=image.shape)
                mask[mask > 0.95] = 0
                mask[mask <= 0.95] = 1
                image = mask * image

            p = random.random()
            if p > 0.5:  
                angle = random.randint(-20, 20)
                image = transforms.functional.rotate(image, angle=angle)
                angle = -angle
                idx = target == -1
                angle_radians = math.radians(angle)
                x = (target[::2] - 48) * math.cos(angle_radians) - (target[1::2] - 48) * math.sin(angle_radians)
                y = (target[::2] - 48) * math.sin(angle_radians) + (target[1::2] - 48) * math.cos(angle_radians)
                target[::2] = x + 48
                target[1::2] = y + 48
                target[idx] = -1

        if self.mode == 'train' or self.mode == 'val':
            return image, target
        else:
            return image
        
    def __getitem__(self, index):
        if self.mode == 'train' or self.mode == 'val':
            return self.transform(self.images[index], target=self.targets[index])
        else:
            return self.transform(self.images[index])

    def __len__(self):
        return self.images.shape[0]

In [ ]:
def reshape_images(images):
    images_reshaped = np.zeros((images.shape[0], 96, 96, 1))
    for i, img in enumerate(images):
        img = img.split(' ')
        img = np.array([int(num) for num in img])
        img = img.reshape((96, 96, 1))
        images_reshaped[i] = img
    
    return images_reshaped
        
raw = pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip', compression='zip')
raw = raw.sample(frac=1)
features_name = list(raw)   
features_name.remove('Image')
images = raw['Image']
targets = raw.drop(columns=['Image'])
targets = targets.fillna(-1).to_numpy()

train_images = images[0: int(raw.shape[0] * 0.9)]
train_targets = targets[0: int(raw.shape[0] * 0.9)]
train_images = reshape_images(train_images)
    
val_images = images[int(raw.shape[0] * 0.999): ]
val_targets = targets[int(raw.shape[0] * 0.999): ]
val_images = reshape_images(val_images)

raw = pd.read_csv('/kaggle/input/facial-keypoints-detection/test.zip', compression='zip')
test_images = raw['Image']
test_images = reshape_images(test_images)

In [ ]:
loss_func = nn.MSELoss()
batch_size = 64
learning_rate = 0.001
num_epochs = 210
decay_every = 30
decay = 0.25

In [ ]:
train_dataset = MyDataset('train', train_images, train_targets)
val_dataset = MyDataset('val', val_images, val_targets)
test_dataset = MyDataset('test', test_images)
train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
models = [InceptionNextModel(), MobileNetModel(), DenseNetModel(), EfficientNetModel()]

In [ ]:
for model in models:
    all_epochs_train_loss = []
    all_epochs_val_loss = []
    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.1)
    scheduler = StepLR(optimizer, decay_every, gamma=decay)
    for epoch in range(1, num_epochs + 1):
        print('epoch:', epoch)
        model.train()
        for samples, targets in train_loader:
            samples = samples.to(device).float()
            targets = targets.to(device).float()
            preds = model(samples)
            idx = targets == -1
            preds[idx] = -1
            loss = loss_func(preds, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 

        scheduler.step()
        model.eval() 
        with torch.no_grad():
            for mode, loader in zip(['train', 'val'], [train_loader, val_loader]):
                epoch_loss, num_samples = 0, 0
                for samples, targets in loader:
                    samples = samples.to(device).float()
                    targets = targets.to(device).float()
                    preds = model(samples)
                    idx = targets == -1
                    preds[idx] = -1
                    loss = loss_func(preds, targets)
                    epoch_loss += loss.item() * targets.shape[0] 
                    num_samples += targets.shape[0]

                epoch_loss = np.sqrt(epoch_loss / num_samples)
                if mode == 'train':
                    all_epochs_train_loss.append(epoch_loss)
                else:
                    all_epochs_val_loss.append(epoch_loss)
                print(mode, '- loss:', f'{epoch_loss:.4}')
            
    plt.plot(np.arange(num_epochs), all_epochs_train_loss, np.arange(num_epochs), all_epochs_val_loss)
    plt.show()

In [ ]:
all_locs = []
for model in models:
    loc = []
    model.eval() 
    with torch.no_grad():
        for samples in test_loader:
            samples = samples.to(device).float()
            preds = model(samples)
            loc = loc + preds.flatten().tolist()
    
    all_locs.append(loc)

loc = np.array(all_locs).mean(axis=0)

In [ ]:
df = pd.DataFrame({'ImageId': np.array([[i] * 30 for i in range(1, 1784)]).flatten(),
                   'FeatureName': features_name * 1783,
                   'Location': loc})
sample_sub = pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')
preds = df.merge(sample_sub, on=['ImageId', 'FeatureName'])['Location_x']
sub = pd.read_csv('/kaggle/input/facial-keypoints-detection/SampleSubmission.csv')
sub['Location'] = preds
sub.to_csv('submission.csv',index = False)